# Automating Grading with OpenAI Function Calling

OpenAI's [function calling](https://platform.openai.com/docs/guides/function-calling) allows the AI to determine when to invoke functions based on user input. It formats the required data and sends it to the function, but the execution is handled by the connected application or system.

This tutorial demonstrates how to use function calling to grade students' submissions on Canvas.

## Set up a Database and Request API Keys

1. **Store Canvas LMS Developer Key**  
   Identify the [Canvas LMS develop key](https://canvas.instructure.com/doc/api/file.developer_keys.html) from your Canvas account and store it in AWS Secrets Manager.  
   - **Key Name**: `api_key`  
   - **Key Value**: Retrieve this information from your Canvas account.  
   - **Secret Name**: `canvas`  
   

2. **Store OpenAI API Key**  
   Purchase and store your [OpenAI](https://openai.com/) API key in AWS Secrets Manager.  
   - **Key Name**: `api_key`  
   - **Key Value**: `<your OpenAI API key>`  
   - **Secret Name**: `openai`

## Install Python Libraries
- **`canvasapi`**: Retrieve Canvas submissions and post grades  
- **`openai`**: Utilize LLMs and function calling  
- **`pymongo`**: Check students' answers  

In [ ]:
pip install openai canvasapi pymongo --quiet

## Secrets Manager Function

In [ ]:
import boto3
from botocore.exceptions import ClientError
import json

def get_secret(secret_name):
    region_name = "us-east-1"

    # Create a Secrets Manager client
    session = boto3.session.Session()
    client = session.client(
        service_name='secretsmanager',
        region_name=region_name
    )

    try:
        get_secret_value_response = client.get_secret_value(
            SecretId=secret_name
        )
    except ClientError as e:
        raise e

    secret = get_secret_value_response['SecretString']
    
    return json.loads(secret)

## Import Python Libraries and Credentials

In [ ]:
import json
import re
import os
from openai import OpenAI
from pprint import pprint
from tqdm.auto import tqdm
from canvasapi import Canvas
import pymongo
from pymongo import MongoClient
from datetime import datetime
from datetime import timezone

# Openai client
openai_api_key  = get_secret('openai')['api_key']
client = OpenAI(api_key=openai_api_key)

# Canvas API URL and key
API_URL = "https://canvas.jmu.edu/"
canvas_api_key = get_secret('canvas')['api_key']

# Initialize Canvas object
canvas = Canvas(API_URL, canvas_api_key)

# Course and assignment IDs
course_id = 2035535  # Replace with your actual course ID
assignment_id = 19255533  # Replace with your actual assignment ID

# This code only process the test student
demo_student_id = 6117320  # the demo student ID


## Utility Functions
- **`retrieve_submissions`**: Retrieve submissions from Canvas  
- **`post_grade`**: Post grades to Canvas  

In [ ]:
def retrieve_submissions():
    try:

        course = canvas.get_course(course_id)

        assignment = course.get_assignment(assignment_id)

        submissions = assignment.get_submissions()
        return submissions

    except Exception as e:
        print(f"An error occurred: {str(e)}")


In [ ]:
def post_grade(course_id, assignment_id, student_id, grade, comment=None):
    try:
        course = canvas.get_course(course_id)

        assignment = course.get_assignment(assignment_id)
        
        #here is to edit a graded submission
        result = submission.edit(submission={'posted_grade': grade}, comment={'text_comment': comment}) 
        
        print(f"Grade posted successfully for student {student_id}")
        return result
   

    except Exception as e:
        print(f"An error occurred: {str(e)}")
        return None

## Calculate grades with a universal correct answer.
Define an `openai_help` function to pass the prompt. 

In [ ]:
from openai import OpenAI

openai_api_key  = get_secret('openai')['api_key']
client = OpenAI(api_key=openai_api_key)
model = 'gpt-4-0613'
temperature = 0

def openai_help(messages, model=model, temperature =temperature ):
    messages = messages
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,

    )
    return response.choices[0].message.content
    # return response 

In the prompt, provide the students' submissions and the correct answers, and ask the AI to grade the submissions.

In [ ]:
delimiter = '###'

right_answer = """
[{'q1 right answer': {'maxSalary': 221900.0}},
 {'q2 right answer': {'_id': 'McLean, Virginia', 'count': 35}},
 {'q3 right answer': {'_id': 'Office of the Director of National Intelligence',
   'job_count': 34}},
 {'q4 right answer': {'total_jobs_starting_in_oct_2024': 102}},
 {'q5 right answer': {'AI_jobs_count': 6}}]
            """


student_grades=[]


for submission in retrieve_submissions():
     if submission.body:
        # print(submission.user_id)
        student_grade={}
        
        #this code only process the test student
        if submission.user_id == demo_student_id:  
            
            student_grade['student_id']=submission.user_id
    
            submisson_text = submission.body
            # print(submisson_text)

            messages = [
                {"role": "system", "content": f"""
                                Students are asked to provide a valid MongoDB connection string and answer five questions:

                                Q1: {delimiter} What is the highest salary in the job data you collected?
                                Q2: {delimiter} Which location has the most jobs?
                                Q3: {delimiter} Which organization posted the most jobs?
                                Q4: {delimiter} How many jobs start in October 2024?
                                Q5: {delimiter} How many jobs mentioned AI in the qualification summary?
                                
                                
                                Analyze the student submissions, delimited by {delimiter}, 
                                and compare them to the correct answers, also delimited by {delimiter}. 
                                The total submission is worth 100 points,
                                and each question is worth 20 points. 
                                Each correct answer receives 20 points, 
                                each wrong answer receives 10 points,
                                and no answer receives 0 points. 
                                If a student fails to provide a connection string, 
                                they receive 0 for the assignment.
                                Calculate the total score and provide a comment to
                                explain why they lost points in a JSON document with keys <score> and <comment>."""},
                
                {"role": "user", "content": f"""student submission:{delimiter}{submisson_text},
                                             right ansers: {delimiter}{right_answer}{delimiter}"""},

                ]
            
            # print(openai_help(messages))
            
            student_grade['grade'] = json.loads(openai_help(messages))
            student_grades.append(student_grade)

Post the grade and comments to Canvas.

In [ ]:
for student_grade in student_grades:
    
    pprint(student_grade)
    post_grade(course_id, assignment_id, student_grade['student_id'], grade= student_grade['grade']['score'], comment=student_grade['grade']['comment'])

## Calculate grades with a different correct answer.
Define a `check_answer` function to check students' submissions with the provided connection string.

In [ ]:
def check_answer(connection_string):

    # check connection string 
    checked_answer = {}
    try:
        client = MongoClient(connection_string)
        checked_answer['connection_string']=connection_string

    except Exception  as e:
        checked_answer['connection_string']=e
        return (checked_answer)
   
    #Q1 answer
    
    try:
        q1_result =  client['demo']['job_collection'].aggregate([
            {
                '$unwind': '$PositionRemuneration'
            }, {
                '$group': {
                    '_id': None, 
                    'maxSalary': {
                        '$max': {
                            '$toDouble': '$PositionRemuneration.MaximumRange'
                        }
                    }
                }
            }, {
                '$project': {
                    '_id': 0, 
                    'maxSalary': 1
                }
            }
        ])

        checked_answer['q1']=next(q1_result)
    except Exception  as e:
        checked_answer['q1 is wrong']=e
    
    #Q2 answer

    try:
        q2_result =  client['demo']['job_collection'].aggregate([
                {
                    '$unwind': '$PositionLocation'
                }, {
                    '$group': {
                        '_id': '$PositionLocation.LocationName', 
                        'count': {
                            '$sum': 1
                        }
                    }
                }, {
                    '$sort': {
                        'count': -1
                    }
                }, {
                    '$limit': 1
                }
            ])

        checked_answer['q2']=next(q2_result)
    except Exception  as e:
        checked_answer['q2 is wrong']=e
        
    #Q3 answer
    
    try:
        q3_result =   client['demo']['job_collection'].aggregate([
        {
            '$group': {
                '_id': '$OrganizationName', 
                'job_count': {
                    '$sum': 1
                }
            }
        }, {
            '$sort': {
                'job_count': -1
            }
        }, {
            '$limit': 1
        }
    ])

        checked_answer['q3']=next(q3_result)
    except Exception  as e:
        checked_answer['q3 is wrong']=e
        
        
    #Q4 answer
    
    try:
        q4_result = client['demo']['job_collection'].aggregate([
            {
                '$addFields': {
                    'PositionStartDate': {
                        '$toDate': '$PositionStartDate'
                    }
                }
            }, {
                '$match': {
                    'PositionStartDate': {
                        '$gte': datetime(2024, 10, 1, 0, 0, 0, tzinfo=timezone.utc), 
                        '$lt': datetime(2024, 11, 1, 0, 0, 0, tzinfo=timezone.utc)
                    }
                }
            }, {
                '$count': 'total_jobs_starting_in_oct_2024'
            }
        ])

        checked_answer['q4']=next(q4_result)
    except Exception  as e:
        checked_answer['q4 is wrong']=e

   
    # Q5 answer
    try:
        q5_result = client['demo']['job_collection'].aggregate([
            {
                '$match': {
                    '$text': {
                        '$search': 'AI'
                    }
                }
            }, {
                '$count': 'AI_jobs_count'
            }
        ])
    
        checked_answer['q5']=next(q5_result)
    except Exception  as e:
        checked_answer['q5 is wrong']=e
        
    # print(checked_answer)
    return(checked_answer)


In [ ]:
connection_string = ''

check_answer(connection_string)

Write the **function schema**. The following schema is generated using the  [OpenAI Function calling guide](https://platform.openai.com/docs/guides/function-calling).

In [ ]:
functions = [
    {
        "name": "check_answer",
        "description": "Checks student responses based on a provided MongoDB connection string.",
        "parameters": {
            "type": "object",
            "properties": {
                "connection_string": {
                    "type": "string",
                    "description": "MongoDB connection string to check the student's answers."
                }
            },
            "required": ["connection_string"]
        }
    }
]


Define an `openai_help_function` to handle prompts and the `check_answer` function.

In [ ]:
from openai import OpenAI

openai_api_key  = get_secret('openai')['api_key']
client = OpenAI(api_key=openai_api_key)
model = 'gpt-4-0613'
temperature = 0

def openai_help_function(messages, model=model, temperature =temperature ):
    messages = messages
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
        functions = functions
    )
    return response 

Write two prompts:

1. The first extracts the students' submissions and `connection string`, and if the `connection string` is provided in the student's submission, call the `check_answer` function.
2. If the function call is provided, pass the extracted parameters to the `check_answer` function, retrieve the correct answer, compare it, and return the grades and comments.

In [ ]:
delimiter = '###'
student_grades=[]
for submission in retrieve_submissions():
     if submission.body:
        # print(submission.user_id)
        student_grade={}
        
        #this code only checks the test student
        if submission.user_id == demo_student_id:  

            
            messages = [ {"role": "system", "content": f"""
                        Students are asked to provide a valid MongoDB connection string and answer five questions:

                        Q1: {delimiter} What is the highest salary in the job data you collected?
                        Q2: {delimiter} Which location has the most jobs?
                        Q3: {delimiter} Which organization posted the most jobs?
                        Q4: {delimiter} How many jobs start in October 2024?
                        Q5: {delimiter} How many jobs mentioned AI in the qualification summary?
                        Analyze the student submissions, delimited by {delimiter}, 
                        extract answers to the five questions into a JSON document, 
                        extract the connection string and check the correct answers using the  connection string.
                                                                """},

            {"role": "user", "content":f"""student submission:{delimiter}{submisson_text}"""},            ]

            response= openai_help_function(messages)
            # pprint(response.choices[0])
            
            
            if response.choices[0].finish_reason == 'function_call':
                function_call = response.choices[0].message.function_call
                function_name = function_call.name
                arguments = function_call.arguments
                student_answer = response.choices[0].message.content
                # Call the appropriate function
                if function_name == "check_answer":
                    import json
                    args = json.loads(arguments)
                    right_answer = check_answer(**args)
                    student_grade['student_id']= submission.user_id
                    messages = [
                                        {"role": "system", "content": f"""
                                Students are asked to provide a valid MongoDB connection string and answer five questions:

                                Q1: {delimiter} What is the highest salary in the job data you collected?
                                Q2: {delimiter} Which location has the most jobs?
                                Q3: {delimiter} Which organization posted the most jobs?
                                Q4: {delimiter} How many jobs start in October 2024?
                                Q5: {delimiter} How many jobs mentioned AI in the qualification summary?
                                The student submission and correct answers are delimited by {delimiter}. 
                                
                                
                                Compare the student's answers against the correct answers.
                                The total submission is worth 100 points, and each question is worth 20 points.
                                Each correct answer receives 20 points, each wrong answer receives 10 points, 
                                and no answer receives 0 points.
                                If a student fails to provide a connection string, they receive 0 for this assignment.

                                Calculate the total score and provide a comment to explain the score. 
                                Return the score and comment in a JSON document with keys <score> and <comment>.
                                            """},
                            {"role": "user", "content": f"""student answer: {delimiter}{student_answer}{delimiter},
                                                         right ansewr: {delimiter}{right_answer}{delimiter}"""},
                                ]
                    # print(openai_help(messages))
                    student_grade['grade'] = json.loads(openai_help(messages))
                    student_grades.append(student_grade)


    

Post the grades and comments to Canvas.

In [ ]:
for student_grade in student_grades:
    
    pprint(student_grade)
    post_grade(course_id, assignment_id, student_grade['student_id'], grade= student_grade['grade']['score'], comment=student_grade['grade']['comment'])

## Reference

- Harrison Chase. *“Functions, Tools and Agents with LangChain.”* DeepLearning.AI. Accessed November 19, 2024. https://www.deeplearning.ai/short-courses/functions-tools-agents-langchain/.

